# Impacto de la deforestación en el regimen de caudales de los rios en Colombia (TEAM 28)

## Multivariate time series forecasting - VAR model

In this notebook we forecast river flow for every month in years 2020-2021, based on precipitation scenarios and expected deforestation trend

### Libraries

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller


### Read data for model training

In [5]:
macrodata = pd.read_csv('../data/matrix/matrix_consol_v2.zip')

In [6]:
data = macrodata.iloc[:, 3:7]

mcs = data['mc'].unique()
mcs.sort()

print(mcs)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]


### Read data for forecast 2020-2021


In [49]:
#data for predictions:
df_2020_2021 = pd.read_csv('../data/matrix/Esc_Predicciones.csv', parse_dates=True, index_col='Date', dayfirst=True)
df_2020_2021['Date'] = df_2020_2021.index
df_2020_2021['month_year'] = pd.to_datetime(df_2020_2021['Date']).dt.to_period('M')
df_2020_2021.head(13)

,MC,P_A1,P_A2,P_B1,P_B2,Def_25%,Def_50%,Def_75%,Def_100%,Date,month_year
Date,,,,,,,,,,,
2020-01-01,1,210.44,202.07,222.87,202.16,0.07693,0.07702,0.07710,0.07718,2020-01-01,2020-01
2020-02-01,1,135.49,130.10,143.49,130.16,0.07702,0.07718,0.07735,0.07751,2020-02-01,2020-02
2020-03-01,1,259.40,249.08,274.72,249.19,0.07710,0.07735,0.07759,0.07784,2020-03-01,2020-03
2020-04-01,1,61.71,59.25,65.35,59.28,0.07718,0.07751,0.07784,0.07817,2020-04-01,2020-04
2020-05-01,1,202.86,194.79,214.84,194.88,0.07726,0.07768,0.07809,0.07850,2020-05-01,2020-05
2020-06-01,1,137.33,131.87,145.44,131.93,0.07735,0.07784,0.07834,0.07883,2020-06-01,2020-06
2020-07-01,1,14.20,13.64,15.04,13.64,0.07743,0.07801,0.07858,0.07916,2020-07-01,2020-07
2020-08-01,1,20.29,19.48,21.49,19.49,0.07751,0.07817,0.07883,0.07949,2020-08-01,2020-08
2020-09-01,1,3.46,3.32,3.66,3.32,0.07759,0.07834,0.07908,0.07982,2020-09-01,2020-09


### Read model results to get the VAR order (p)

In [12]:
df_models = pd.read_csv('VAR_results_2years.csv')
df_models.head()

,basin,mean,stdev,AIC,BIC,RMSE,MAPE,sMAPE,p
0,1,2.890417,0.903569,-9.987177,-9.987177,0.380017,12.022449,5.894861,12
1,2,23.965833,14.174385,-6.680651,-6.680651,10.450333,83.975023,22.543149,12
2,3,151.052083,66.239974,-4.599209,-4.599209,45.509149,31.499597,12.715241,12
3,4,5.925417,2.444201,-8.063515,-8.063515,2.678722,51.019828,18.507438,5
4,5,61.653750,19.756854,-6.976881,-6.976881,22.264741,22.254400,12.930103,5


In [21]:
int(df_models[df_models['basin']==1]['p'])

12

# All basins

In [14]:
from numpy import mean 
from numpy import std
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_1d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc# show inverted results in a dataframe

In [137]:
#P_A1, Def25%
deforest = 'Def_25%'
precip = 'P_A1'
list_df_plot = []

for i in mcs:
    print("\n ################ Basin " + str(i) + " ############### \n")
    #select data for that basin, split train and test
    df_train = data[data['mc'] == i][['v_flow_mean', 'v_loss_cover', 
                                     'v_rainfall_total']].copy().reset_index(drop = True)
    df_test = df_2020_2021[df_2020_2021['MC']==i][[deforest, precip]]
    df_test.rename(columns={deforest: "v_loss_cover", precip: "v_rainfall_total"})
    #print(df_test)
    
    order_p = int(df_models[df_models['basin']==i]['p'])
    #print(order_p)

    # 1st difference to transform to Stationary
    df_differenced = df_train.diff().dropna()# stationarity test again with differenced data


    # VAR model fitting
    model = VAR(df_differenced)
    results = model.fit(order_p, ic='aic') #choose VAR order p from 0 to maxlags, the one with lowest AIC
    #print(results.summary())

    ### results.plot_forecast(12)
    ### plt.show()

    fevd = results.fevd(5)
    #print(fevd.summary())

    # forecasting
    nobs=24
    pred = results.forecast(results.y, steps=nobs)
    df_forecast = pd.DataFrame(pred, index=temp_df.index[-nobs:], columns=temp_df.columns + '_1d')

    df_results = invert_transformation(df_train, df_forecast, second_diff=False)        
    #df_results.loc[:, ['v_flow_mean_forecast', 'v_loss_cover_forecast', 'v_rainfall_total_forecast']]
    df_results.index = df_2020_2021[df_2020_2021['MC']==i]['month_year']
    df_results.v_flow_mean_1d[df_results.v_flow_mean_1d<=0] = 0.1
    
    
    flow = df_train.v_flow_mean.copy()
    tmp_index = macrodata[data['mc'] == i]['date'].reset_index().date.values
    flow.index = tmp_index
    
    flow = pd.concat([flow, df_results['v_flow_mean_forecast']], axis=0)
    ### flow.plot(figsize=(20,5))
    
    df_results['basin'] = i
    df_results['scenario'] = deforest + '_' + precip
    df_test.index = df_results.index
    df_results = pd.merge(df_results, df_test, left_index=True, right_index=True)
    df_results = df_results.rename({deforest: 'v_loss_cover', precip: 'v_rainfall_total'}, axis=1) 
    
    list_df_plot.append(df_results) 


 ################ Basin 1 ############### 


 ################ Basin 2 ############### 


 ################ Basin 3 ############### 


 ################ Basin 4 ############### 


C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'




 ################ Basin 5 ############### 


 ################ Basin 6 ############### 


 ################ Basin 7 ############### 


 ################ Basin 8 ############### 


C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'




 ################ Basin 9 ############### 


 ################ Basin 10 ############### 


 ################ Basin 11 ############### 


 ################ Basin 12 ############### 


...

C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'



 ################ Basin 42 ############### 


 ################ Basin 43 ############### 


 ################ Basin 44 ############### 


 ################ Basin 45 ############### 


C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'




 ################ Basin 46 ############### 


 ################ Basin 47 ############### 


 ################ Basin 48 ############### 


C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [138]:
results1 = pd.concat(list_df_plot, axis=0)
results1 = results1.drop(columns = ['v_flow_mean_1d','v_loss_cover_1d','v_rainfall_total_1d', 'v_loss_cover_forecast','v_rainfall_total_forecast'], axis=0)
results1

,v_flow_mean_forecast,basin,scenario,v_loss_cover,v_rainfall_total
month_year,,,,,
2020-01,1.755026,1,Def_25%_P_A1,0.07693,210.44
2020-02,1.665493,1,Def_25%_P_A1,0.07702,135.49
2020-03,2.082901,1,Def_25%_P_A1,0.07710,259.40
2020-04,2.711822,1,Def_25%_P_A1,0.07718,61.71
2020-05,3.474700,1,Def_25%_P_A1,0.07726,202.86
...,...,...,...,...,...
2021-08,8318.086078,48,Def_25%_P_A1,0.06767,187.91
2021-09,7057.524315,48,Def_25%_P_A1,0.06774,174.97
2021-10,5153.063785,48,Def_25%_P_A1,0.06781,293.68


In [142]:
list_df_plot = []
for deforest in ['Def_25%', 'Def_50%', 'Def_75%', 'Def_100%']:
    for precip in ['P_A1','P_A2','P_B1','P_B2']:
        print(deforest)
        print(precip)

        for i in mcs:
            print("\n ################ Basin " + str(i) + " ############### \n")
            #select data for that basin, split train and test
            df_train = data[data['mc'] == i][['v_flow_mean', 'v_loss_cover', 
                                             'v_rainfall_total']].copy().reset_index(drop = True)
            df_test = df_2020_2021[df_2020_2021['MC']==i][[deforest, precip]]
            df_test.rename(columns={deforest: "v_loss_cover", precip: "v_rainfall_total"})
            #print(df_test)

            order_p = int(df_models[df_models['basin']==i]['p'])
            #print(order_p)

            # 1st difference to transform to Stationary
            df_differenced = df_train.diff().dropna()# stationarity test again with differenced data


            # VAR model fitting
            model = VAR(df_differenced)
            results = model.fit(order_p, ic='aic') #choose VAR order p from 0 to maxlags, the one with lowest AIC
            #print(results.summary())

            ### results.plot_forecast(12)
            ### plt.show()

            fevd = results.fevd(5)
            #print(fevd.summary())

            # forecasting
            nobs=24
            pred = results.forecast(results.y, steps=nobs)
            df_forecast = pd.DataFrame(pred, index=temp_df.index[-nobs:], columns=temp_df.columns + '_1d')

            df_results = invert_transformation(df_train, df_forecast, second_diff=False)        
            #df_results.loc[:, ['v_flow_mean_forecast', 'v_loss_cover_forecast', 'v_rainfall_total_forecast']]
            df_results.index = df_2020_2021[df_2020_2021['MC']==i]['month_year']
            df_results.v_flow_mean_1d[df_results.v_flow_mean_1d<=0] = 0.1


            flow = df_train.v_flow_mean.copy()
            tmp_index = macrodata[data['mc'] == i]['date'].reset_index().date.values
            flow.index = tmp_index

            flow = pd.concat([flow, df_results['v_flow_mean_forecast']], axis=0)
            ### flow.plot(figsize=(20,5))

            df_results['basin'] = i
            df_results['scenario'] = deforest + '_' + precip
            df_test.index = df_results.index
            df_results = pd.merge(df_results, df_test, left_index=True, right_index=True)
            df_results = df_results.rename({deforest: 'v_loss_cover', precip: 'v_rainfall_total'}, axis=1) 

            list_df_plot.append(df_results) 

Def_25%
P_A1

 ################ Basin 1 ############### 


 ################ Basin 2 ############### 


 ################ Basin 3 ############### 


 ################ Basin 4 ############### 


 ################ Basin 5 ############### 


C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\base\wrapper.py:36: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\



 ################ Basin 6 ############### 


 ################ Basin 7 ############### 


 ################ Basin 8 ############### 


 ################ Basin 9 ############### 



C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\en


 ################ Basin 10 ############### 


 ################ Basin 11 ############### 


 ################ Basin 12 ############### 


 ################ Basin 13 ############### 


 ################ Basin 14 ############### 


...

C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\en


 ################ Basin 42 ############### 


 ################ Basin 43 ############### 


 ################ Basin 44 ############### 


 ################ Basin 45 ############### 


 ################ Basin 46 ############### 



C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\en


 ################ Basin 47 ############### 


 ################ Basin 48 ############### 



C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
C:\Users\Diana\.conda\envs\ds4a\lib\site-packages\statsmodels\tsa\base\tsa_model.py:213: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'


In [143]:
results1 = pd.concat(list_df_plot, axis=0)
results1 = results1.drop(columns = ['v_flow_mean_1d','v_loss_cover_1d','v_rainfall_total_1d', 'v_loss_cover_forecast','v_rainfall_total_forecast'], axis=0)
results1

,v_flow_mean_forecast,basin,scenario,v_loss_cover,v_rainfall_total
month_year,,,,,
2020-01,1.755026,1,Def_25%_P_A1,0.07693,210.44
2020-02,1.665493,1,Def_25%_P_A1,0.07702,135.49
2020-03,2.082901,1,Def_25%_P_A1,0.07710,259.40
2020-04,2.711822,1,Def_25%_P_A1,0.07718,61.71
2020-05,3.474700,1,Def_25%_P_A1,0.07726,202.86
...,...,...,...,...,...
2021-08,8318.086078,48,Def_100%_P_B2,0.07172,193.73
2021-09,7057.524315,48,Def_100%_P_B2,0.07199,180.40
2021-10,5153.063785,48,Def_100%_P_B2,0.07226,302.79


In [144]:
results1.to_csv(r'VAR_forecast_2020_2021.csv')